# Install packages
Reload after installation, then mount drive

In [ ]:
!pip install timm
!pip install pillow==4.1.1
%reload_ext autoreload
%autoreload

# Mount your Drive

In [ ]:
#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

#Navigate to /mydrive/yolov4
%cd /mydrive/DataScienceProject/New/

# list the contents of /mydrive
!ls

# Download MiDas transformer

In [ ]:
# 125 frames to 125 ply = 5 minutes 
import cv2
import torch
import time
import numpy as np
from google.colab.patches import cv2_imshow 
import os
from os import listdir
import glob

# Load a MiDas model for depth estimation
model_type = "DPT_Large"     # MiDaS v3 - Large     (highest accuracy, slowest inference speed)
#model_type = "DPT_Hybrid"   # MiDaS v3 - Hybrid    (medium accuracy, medium inference speed)
#model_type = "MiDaS_small"  # MiDaS v2.1 - Small   (lowest accuracy, highest inference speed)

midas = torch.hub.load("intel-isl/MiDaS", model_type)

# Move model to GPU if available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

# Load transforms to resize and normalize the image
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

Downloading: "https://github.com/intel-isl/MiDaS/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/intel-isl/DPT/releases/download/1_0/dpt_large-midas-2f21e586.pt" to /root/.cache/torch/hub/checkpoints/dpt_large-midas-2f21e586.pt


  0%|          | 0.00/1.28G [00:00<?, ?B/s]

Using cache found in /root/.cache/torch/hub/intel-isl_MiDaS_master


# Implement Q matrix from Intrinsic Camera Parameters

In [ ]:
Q = np.array(([1.0, 0.0, 0.0, -934.5202026367188],
              [0.0, 1.0, 0.0, -444.3987731933594],
              [0.0, 0.0, 0.0, 850.0],
              [0.0, 0.0, 1.0/90.0, 0.0]),dtype=np.float32)

# Create list of frames from clipped video

In [ ]:
# Add paths to folder and img_dir for multiple images
folder = ""
img_dir = f"input/{folder}/" # Enter Directory of all images  
data_path = os.path.join(img_dir,'*.png') 
files = glob.glob(data_path) 

# create list of images from folder
data = [] 
for f1 in files: 
    img = cv2.imread(f1) 
    data.append(img)

# able to get ith image in list
length = len(data)
print(f'Number of frames in folder: {length}')

Number of frames in folder: 1


# Function to create Point Cloud from Depth Map

In [ ]:
# Create folder for ply files
os.mkdir(f'output/Large/{folder}')

In [ ]:
#Function to create point cloud file
def create_output(vertices, colors, filename):
	colors = colors.reshape(-1,3)
	vertices = np.hstack([vertices.reshape(-1,3),colors])

	ply_header = '''ply
		format ascii 1.0
		element vertex %(vert_num)d
		property float x
		property float y
		property float z
		property uchar red
		property uchar green
		property uchar blue
		end_header
		'''
	with open(filename, 'w') as f:
		f.write(ply_header %dict(vert_num=len(vertices)))
		np.savetxt(f,vertices,'%f %f %f %d %d %d')

# Convert Depth map to PLY files

In [ ]:
# Create folder to store saved depth maps
os.mkdir(f'output/Large/DepthMaps_{folder}')

In [ ]:
for i in range(length):
  if i % 1 == 0:
    
    start = time.time()

    depthmaps = f'output/Large/DepthMaps_{folder}/DepthMap_'+str(i)+'.png' # save video frame as png

    # Convert to RGB format
    img = cv2.cvtColor(data[i], cv2.COLOR_BGR2RGB)

    # Apply input transforms
    input_batch = transform(img).to(device)

    # Prediction and resize to original resolution
    with torch.no_grad():
        prediction = midas(input_batch)

        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[:2],
            mode="bicubic",
            align_corners=False,
        ).squeeze()

    depth_map = prediction.cpu().numpy()
    depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

    #Reproject points into 3D
    points_3D = cv2.reprojectImageTo3D(depth_map, Q, handleMissingValues=False)

    #Get rid of points with value 0 (i.e no depth)
    depth_threshhold = 0.2
    mask_map = depth_map > depth_threshhold

    #Mask colors and points. 
    output_points = points_3D[mask_map]
    output_colors = img[mask_map]

    end = time.time()
    totalTime = end - start
    fps = 1 / totalTime

    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    depth_map = (depth_map*255).astype(np.uint8)
    depth_map = cv2.applyColorMap(depth_map , cv2.COLORMAP_MAGMA)
    #cv2_imshow(depth_map)
    cv2.imwrite(depthmaps,depth_map) # save Depth Map frame
    #Generate point cloud 
    #output_file = f'output/Large/{folder}/{folder}-' + str(depth_threshhold) + '_' + str(i)+ '.ply'
    output_file = f'output/Large/{folder}/{folder}_' + str(i)+ '.ply'
    create_output(output_points, output_colors, output_file)
    
    # Progress check
    remaining = length - i
    if i % 25 == 0:
      print(f'Progress: {i} point cloud files saved successfully.\n {remaining} remaining until finished.')
    else:
      continue 
  else:
    continue

Progress: 0 point cloud files saved successfully.
 1 remaining until finished.
